## 1.0 - The first task takes the given dataset and train a word2vec model on it 

In [1]:
import pandas as pd
import pathlib, glob, time, subprocess, re, pygraphviz, json
from tabulate import tabulate
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import networkx as nx
import scipy.sparse as sp
import numpy as np
from src.utils.functions.parse import tokenizer
import src.data as data_util

In [2]:
#location to csv file, in our case it is MSR_data
new_data  = pd.read_csv('msr/MSR_data_cleaned.csv')
new_data_source = pd.DataFrame(data={'func':pd.concat([new_data['func_before'], new_data['func_after']])}) 

/home/ammar/vuln_code/vuln_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (20,22,23,27,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
tokens_dataset = data_util.tokenize(new_data_source.iloc[0:100])

/home/ammar/vuln_code/vuln_env/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
print('Tokenizing source code')
tokens_dataset = data_util.tokenize(new_data_source)

#you can change here that instead of training word2vec again load the available dataset
print('Training word2vec on tokens')
w2vmodel = Word2Vec(sentences=tokens_dataset.tokens, vector_size=200, window=5, min_count=1, workers=4, epochs=10)
print('Saving word2vec model')
w2vmodel.save('msr/all_w2v_model_devign_tokens.model')
#load it using: Word2Vec.load('msr/all_w2v_model_devign_tokens.model')

In [ ]:
sample_size = 11000
new_data_pos = new_data[new_data['vul']==1]['func_before']
new_data_neg = new_data[new_data['vul']==0]['func_after'].sample(sample_size)

from pathlib import Path
curr_path = "new_dataset_update/positive" 
Path(curr_path).mkdir(parents=True, exist_ok=True)
curr_path = "new_dataset_update/negative"
Path(curr_path).mkdir(parents=True, exist_ok=True)

#writing the code from pandas to individual .c files
for key, c_code in new_data_pos.items():
    with open('new_dataset_update/positive/{}.c'.format(key), 'w') as f:
        f.write(c_code)
        
for key, c_code in new_data_neg.items():
    with open('new_dataset_update/negative/{}.c'.format(key), 'w') as f:
        f.write(c_code)

## 2.0 - The following code convert the C-files in the given directory to cfg representation 

In [ ]:
# change the given three variable according to your environment
graph_type = 'cfg'
path_code = pathlib.Path('new_dataset_update/')   #where c code is present in the folder                    
joern_path = '/home/ammar/vuln_code/joern-cli/'   #local path where joern-cli is present

# iterate over each group/project
for folder in tqdm(glob.glob('{}/*'.format(path_code))): #the group/folders in the directory are
                                                            #negative and positive in this case
    if graph_type not in folder and Path(folder).is_dir():
        print(folder)
        path_group = pathlib.Path('{}_{}'.format(folder,graph_type))         #make folder for the dataset
        path_group.mkdir(parents=True, exist_ok=True)
    
        start_time = time.time()

        project = glob.glob(folder+'/*')                  #get all .c files we just generated

        for path_src in tqdm(project):
            src_file      = str(Path(path_src).resolve())
            out_path      = str(Path(str(path_group)+'/'+path_src.split('/')[-1].replace('.c','')).resolve())+"/"
            file_dir      = pathlib.Path(str(Path(str(path_group)+'/'+path_src.split('/')[-1].replace('.c','')).resolve())+"/")         #make folder for the dataset
            file_dir.mkdir(parents=True, exist_ok=True)

            cpg_path_cmd  = [joern_path+"./joern-parse",src_file,"--out",out_path+src_file.split("/")[-1].replace(".c","")+".cpg"]
            graph_out_cmd = [joern_path+"joern-export",out_path+src_file.split("/")[-1].replace(".c","")+'.cpg',"--repr",graph_type,'--out',out_path+'/cfg/']
            cpg_check     = Path(out_path+src_file.split("/")[-1].replace(".c","")+".cpg").is_file()
            
            pth = pathlib.Path(str(Path(str(path_group)+'/'+path_src.split('/')[-1].replace('.c','')).resolve())+"/")
            if cpg_check and not Path(out_path+'cfg/').is_dir():
                result = subprocess.call(graph_out_cmd,stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL)

            # check if CFG and both CPG are present if true nothing will be generated
            elif cpg_check and Path(out_path+'cfg/').is_dir():
                pass

            else:
            #if both of them all false compute CPG and generate CFG 
                result = subprocess.call(cpg_path_cmd,stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL)

                result1 = subprocess.call(graph_out_cmd, stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL)
                
            with open(out_path+pth.name+'.c', 'w') as f, open(src_file, 'r') as f1:
                f.write(f1.read())

        print("--- %s miuntes ---" % ((time.time() - start_time)/60))

## 3.0 - Converting the CFG representation into networkx graph 

In [ ]:
check_empty  ='digraph .*\{ *\n*\}' #to check if generated CFG is empty or not
pd_list = []
dict_e = {}
path_code = 'new_dataset_update' #I have changed the name
project_path = glob.glob(path_code+"/*_cfg")
for i in tqdm(project_path):
    path = pathlib.PurePath(i)
    project_name = path.name 
    project = glob.glob(i+'/*')
    dict_e[i+' total'] = len(project)
    dict_e[i+' removed'] = 0
    for path_src in tqdm(project, 'total files'):
        #print(path_src)
        new_dict   = dict()
        src_file   = str(Path(path_src+'/'+pathlib.PurePath(path_src).name+'.c').resolve())
        cfg_folder = str(Path(path_src+'/'+'cfg/').resolve())
        index      = int(pathlib.PurePath(src_file).name.replace(".c",""))
        target     = project_name.split('_')[0]
        #src_file   =  str(Path(os.path.join('', *[path_code,target,str(index)+'.c'])).resolve()) #changed for new_Dataset
        #print('{}- {}- {}- {}- {}'.format(path_src, cfg_folder,index,target,src_file))
        with open(src_file, 'r') as f:
            src_code = f.read()
        dot_arr = []
        for file in glob.glob(cfg_folder+"/*"):
            with open(file,'r') as f:
                dot_arr.append(f.read())
        dot_arr = [x for x in dot_arr if not re.search(check_empty, x)] # removes empty graphs
        if (len(dot_arr) == 1):                                         #if graph is not empty and is connected
            is_connected = True
            #G = nx.Graph(nx.drawing.nx_pydot.read_dot(Path(cfg_folder).joinpath("0-cfg.dot")))
            try:
                with open(Path(cfg_folder).joinpath("0-cfg.dot")) as f:
                    dotFormat = f.read()
                new_str = dotFormat.replace('\\"', '')                       #To catch escape characters
                new_str = "\n".join([f_str.strip() for f_str in dotFormat.split('\n')])
                G = nx.drawing.nx_agraph.from_agraph(pygraphviz.AGraph(new_str)); #convert graph into Networkx object
                new_dict['index']         = index
                new_dict['project']       = project_name
                new_dict['func_code']     = src_code
                new_dict['graph']         = G
                new_dict['is_connected']  = is_connected
                new_dict['dot_string']    = new_str
                if target=='positive':
                    new_dict['target']        = 1
                elif target =='negative':
                    new_dict['target']        = 0
                else:
                    print('class other than negative and positive found')
                pd_list.append(new_dict)
            except ValueError as e:
                dict_e[i+' removed'] += 1
                pass
            
        dict_e[i+' used'] = dict_e[i+' total'] - dict_e[i+' removed']

## 3.1 - Displays basic stats of the graph dataset

In [ ]:
print(json.dumps(dict_e, indent=4))
data = pd.DataFrame(pd_list)
data = data[['target', 'project', 'graph','func_code','index']]
data = data.rename(columns={'func_code': 'func'})
print('tokenizing code')
w2vmodel = Word2Vec.load('new_dataset_update/word2vec.model')

print("No of samples in dataset: {} ".format(len(data)))
print('Complete dataset stats')
print('\n'*3,'Complete dataset stats\n',"*"*40,'\n')

node_size_group = data.apply(lambda g: nx.number_of_nodes(g.graph),axis=1).describe()[['min', 'max','mean','std']]

print(tabulate(node_size_group.to_frame(),
               tablefmt="grid", stralign='left', numalign='left',
               headers=['Node stats']))


print('\n'*3)

edge_size_group = data.apply(lambda g: nx.number_of_edges(g.graph),axis=1).describe()[['min', 'max','mean','std']]
print(tabulate(edge_size_group.to_frame(),
                   tablefmt="grid", stralign='left', numalign='left',
                   headers=['Edge stats']))

print('\n\n')
print('Stats for each class')
print("1 = Vulnerable, 0 = Not Vulnerable")
print('\n')
for name, group in data.groupby('target'):
    
    edge_size_group = group.apply(lambda g: nx.number_of_edges(g.graph),axis=1).describe()[['min', 'max','mean','std']]
    node_size_group = group.apply(lambda g: nx.number_of_nodes(g.graph),axis=1).describe()[['min', 'max','mean','std']]
    
    print(tabulate(node_size_group.to_frame(),
               tablefmt="grid", stralign='left', numalign='left',
               headers=['Class {} Node stats'.format(name)]))
    
    print('\n'*3)
    print(tabulate(edge_size_group.to_frame(),
                   tablefmt="grid", stralign='left', numalign='left',
                   headers=['Class {} Edge stats'.format(name)]))
    print('\n'*3)

## 3.2 - Saves the dataset in pytorch geometric

In [ ]:
data["input"] = data.apply(lambda row: nodes_to_input(row.graph, row.target, nx.number_of_nodes(row.graph),
                                                                                    w2vmodel.wv), axis=1)
print('Writing to file/pandas')
pd.to_pickle(data[['input','target']], 'new_dataset_update/data_random.pkl')